In [1]:
from datasets import load_dataset
import pandas as pd

In [2]:
trueteacher = load_dataset("google/trueteacher")

In [3]:
cnn_dailymail = load_dataset("cnn_dailymail", "3.0.0")

In [4]:
df = trueteacher['train'].to_pandas()

df['label'] = df['label'].astype(int)

df['summarization_model'].value_counts()

summarization_model
T5-Base     279657
T5-Small    278141
T5-Large    277667
T5-3B       274759
T5-11B      273238
Name: count, dtype: int64

In [5]:
subset1 = df[df['summarization_model'] == 'T5-3B']
subset2 = df[df['summarization_model'] == 'T5-11B']
subset3 = df[df['summarization_model'] == 'T5-Large']

subset = pd.concat([subset2, subset1, subset3])
len(subset)

825664

In [6]:
negatives = subset[subset['label'] == 0].drop_duplicates(subset=['cnndm_id'], keep="first")
positives = subset[subset['label'] == 1].drop_duplicates(subset=['cnndm_id'], keep="first")

uniques = pd.concat([negatives, positives.sample(n=len(negatives))])

print(len(negatives)), print(len(positives)), print(len(uniques))

124772
273245
249544


(None, None, None)

In [7]:
len(set(negatives['cnndm_id'].to_list())), len(set(positives['cnndm_id'].to_list()))

(124772, 273245)

In [8]:
cnn_dailymail = cnn_dailymail['train'].to_pandas()
cnn_dailymail.head(2)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9


In [9]:
# join with the following keys cnndm_id on subset and id on cnn_dailymail

merged = pd.merge(uniques, cnn_dailymail, left_on='cnndm_id', right_on='id')
len(merged)

249544

In [10]:
merged = merged[['article', 'summary', 'label', 'summarization_model']]

merged = merged.rename(columns={'summary': 'target', 'article': 'source', 'label': 'label', 'summarization_model': 'subset'})


# Replace 1 with Yes and 0 with No
merged['label'] = merged['label'].replace({1: 'Yes', 0: 'No'})

merged

,source,target,label,subset
0,"By . Anthony Bond . PUBLISHED: . 12:26 EST, 12...",A man who served 39 years in prison for the ra...,No,T5-11B
1,The production is slick. The imagery: ominous....,Amid the escalating violence in Iraq and Syria...,No,T5-11B
2,(CNN) -- Tayna Fogle sat just behind Sen. Rand...,A former felon who helped launch a US presiden...,No,T5-11B
3,An Iraqi policeman has been hailed a hero afte...,The UK Ministry of Defence (MoD) has said it i...,No,T5-11B
4,By . Simon Jones . West Ham are waiting on Che...,West Ham manager Slaven Bilic has been given p...,No,T5-11B
...,...,...,...,...
249539,Queens Park Rangers caretaker manager Chris Ra...,QPR caretaker manager Chris Ramsey says he wou...,Yes,T5-11B
249540,A mother of two in South Carolina has been cuf...,A US mother was arrested for swearing in front...,Yes,T5-11B
249541,"Karin Ward, 56, is facing a High Court trial ...",A woman who claims she was sexually abused by ...,Yes,T5-11B
249542,"By . Kerry Mcdermott . PUBLISHED: . 09:01 EST,...",A Chinese ship is attempting to make history b...,Yes,T5-11B


In [11]:
merged.to_json('../data/test-data/trueteacher_unique_sampled.json', orient='records', lines=True, force_ascii=False)

In [2]:
#load 

df = pd.read_json('../data/test-data/trueteacher_unique_sampled.json', orient='records', lines=True)
df

,source,target,label,subset
0,"By . Anthony Bond . PUBLISHED: . 12:26 EST, 12...",A man who served 39 years in prison for the ra...,No,T5-11B
1,The production is slick. The imagery: ominous....,Amid the escalating violence in Iraq and Syria...,No,T5-11B
2,(CNN) -- Tayna Fogle sat just behind Sen. Rand...,A former felon who helped launch a US presiden...,No,T5-11B
3,An Iraqi policeman has been hailed a hero afte...,The UK Ministry of Defence (MoD) has said it i...,No,T5-11B
4,By . Simon Jones . West Ham are waiting on Che...,West Ham manager Slaven Bilic has been given p...,No,T5-11B
...,...,...,...,...
249539,Queens Park Rangers caretaker manager Chris Ra...,QPR caretaker manager Chris Ramsey says he wou...,Yes,T5-11B
249540,A mother of two in South Carolina has been cuf...,A US mother was arrested for swearing in front...,Yes,T5-11B
249541,"Karin Ward, 56, is facing a High Court trial ...",A woman who claims she was sexually abused by ...,Yes,T5-11B
249542,"By . Kerry Mcdermott . PUBLISHED: . 09:01 EST,...",A Chinese ship is attempting to make history b...,Yes,T5-11B


In [13]:
df.subset.value_counts()

subset
T5-11B      146173
T5-Large     54037
T5-3B        49334
Name: count, dtype: int64

In [3]:
def count_words(text):
    return len(text.split())


df['source_word_count'] = df['source'].apply(count_words)
df['target_word_count'] = df['target'].apply(count_words)

In [5]:
df['total_word_count'] = df['source_word_count'] + df['target_word_count']

df['total_word_count'].describe()

count    249544.000000
mean        724.446631
std         345.503751
min          37.000000
25%         466.000000
50%         666.000000
75%         919.000000
max        2374.000000
Name: total_word_count, dtype: float64

In [6]:
df['total_word_count'].sum()

180781310

In [7]:
180781310*0.25

45195327.5

In [8]:
45195327/1000000

45.195327